In [2]:
import pandas as pd

In [9]:
data = pd.read_table('data/XYZCorp_LendingData.txt')
data.to_csv('data/XYZCorp_LendingData.csv', index = False)

C:\Users\Caio\AppData\Local\Temp\ipykernel_10544\3180723633.py:1: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_table('data/XYZCorp_LendingData.txt')


In [10]:
data = pd.read_csv('data/XYZCorp_LendingData.csv')

C:\Users\Caio\AppData\Local\Temp\ipykernel_10544\1445386200.py:1: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/XYZCorp_LendingData.csv')


In [41]:
data[member_info]

,emp_title,emp_length,home_ownership,annual_inc,verification_status,zip_code,addr_state,dti,dti_joint,delinq_2yrs,...,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,pub_rec,mths_since_last_major_derog,annual_inc_joint,dti_joint,verification_status_joint,inq_fi,inq_last_12m
0,NaN,10+ years,RENT,24000.0,Verified,860xx,AZ,27.65,NaN,0.0,...,1.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,Ryder,< 1 year,RENT,30000.0,Source Verified,309xx,GA,1.00,NaN,0.0,...,5.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,10+ years,RENT,12252.0,Not Verified,606xx,IL,8.72,NaN,0.0,...,2.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
3,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,917xx,CA,20.00,NaN,0.0,...,1.0,35.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,University Medical Group,1 year,RENT,80000.0,Source Verified,972xx,OR,17.94,NaN,0.0,...,0.0,38.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
855964,Office Assistant,8 years,RENT,31000.0,Verified,905xx,CA,28.69,NaN,0.0,...,0.0,NaN,77.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
855965,Director of Maintenance,10+ years,MORTGAGE,79000.0,Verified,072xx,NJ,3.90,NaN,0.0,...,1.0,26.0,NaN,0.0,29.0,NaN,NaN,NaN,NaN,NaN
855966,Radiologist Technologist,5 years,RENT,35000.0,Verified,378xx,TN,30.90,NaN,0.0,...,0.0,NaN,84.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
855967,Painter,1 year,RENT,64400.0,Source Verified,010xx,MA,27.19,NaN,1.0,...,2.0,22.0,NaN,0.0,22.0,NaN,NaN,NaN,NaN,NaN


In [40]:
member_info = ['emp_title', 'emp_length', 'home_ownership', 'annual_inc',
               'verification_status', 'zip_code', 'addr_state', 'dti',
               'dti_joint', 'delinq_2yrs', 'earliest_cr_line',
               'inq_last_6mths', 'mths_since_last_delinq',
               'mths_since_last_record', 'pub_rec', 'mths_since_last_major_derog',
               'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'inq_fi', 'inq_last_12m']

account_info = ['open_acc', 'revol_bal', 'revol_util', 'total_acc']

application_info = ['desc', 'purposse', 'title', 'application_type']

loan_info = ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term',
             'int_rate', 'installment', 'grade', 'sub_grade', 'issue_d',
             'pymnt_plan', 'initial_list_status', 'out_prncp', 'out_prncp_inv',
             'total_pymnt', 'total_pymnt_inv', 'totalrecprncp', 'total_rec_int',
             'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
             'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d',
             'last_credit_pull_d', 'policy_code', 'default_ind']

other_info = ['collections_12_mths_ex_med', 'tot_coll_amt', 'acc_now_delinq',
              'totcurbal', 'open_acc_6m', 'open_il_6m', 'open_il_12m', 'open_il_24m',
              'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m',
              'max_bal_bc', 'all_util', 'total_rev_hi_limv', 'total_cu_tl']


In [26]:
for i in data.columns:
    print(i)

id
member_id
loan_amnt
funded_amnt
funded_amnt_inv
term
int_rate
installment
grade
sub_grade
emp_title
emp_length
home_ownership
annual_inc
verification_status
issue_d
pymnt_plan
desc
purpose
title
zip_code
addr_state
dti
delinq_2yrs
earliest_cr_line
inq_last_6mths
mths_since_last_delinq
mths_since_last_record
open_acc
pub_rec
revol_bal
revol_util
total_acc
initial_list_status
out_prncp
out_prncp_inv
total_pymnt
total_pymnt_inv
total_rec_prncp
total_rec_int
total_rec_late_fee
recoveries
collection_recovery_fee
last_pymnt_d
last_pymnt_amnt
next_pymnt_d
last_credit_pull_d
collections_12_mths_ex_med
mths_since_last_major_derog
policy_code
application_type
annual_inc_joint
dti_joint
verification_status_joint
acc_now_delinq
tot_coll_amt
tot_cur_bal
open_acc_6m
open_il_6m
open_il_12m
open_il_24m
mths_since_rcnt_il
total_bal_il
il_util
open_rv_12m
open_rv_24m
max_bal_bc
all_util
total_rev_hi_lim
inq_fi
total_cu_tl
inq_last_12m
default_ind


In [24]:
member_data

,addr_state,annual_inc,annual_inc_joint,application_type
0,AZ,24000.0,NaN,INDIVIDUAL
1,GA,30000.0,NaN,INDIVIDUAL
2,IL,12252.0,NaN,INDIVIDUAL
3,CA,49200.0,NaN,INDIVIDUAL
4,OR,80000.0,NaN,INDIVIDUAL
...,...,...,...,...
855964,CA,31000.0,NaN,INDIVIDUAL
855965,NJ,79000.0,NaN,INDIVIDUAL
855966,TN,35000.0,NaN,INDIVIDUAL
855967,MA,64400.0,NaN,INDIVIDUAL


In [12]:
data = pd.read_table('data/XYZCorp_LendingData.txt')

C:\Users\Caio\AppData\Local\Temp\ipykernel_10544\3312923805.py:1: DtypeWarning: Columns (17,45,53) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_table('data/XYZCorp_LendingData.txt')


In [20]:
print(data.columns)

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'pymnt_plan', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint',
       'verification_status_joint', 'acc_now_delinq', 'tot_col

In [7]:
data['term'].value_counts()

 36 months    600221
 60 months    255748
Name: term, dtype: int64